In [ ]:
import sys

MI_PATH = "/home/leroyv/Documents/src/thirdparty/mitsuba3/build/python"

if MI_PATH not in sys.path:
    sys.path.insert(0, MI_PATH)
    display(sys.path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mitsuba as mi
import drjit as dr
import seaborn as sns

#dr.set_thread_count(23)
mi.set_variant("scalar_rgb_double")
mi_logger = mi.Thread.thread().logger()
sns.set_theme(style="ticks", rc={"image.cmap": "viridis"}, context="talk")

In [ ]:
n = 4096  # Number of grid points
zmax = 1.0  # Maximum height
tau = 1.0  # Total Optical thickness
l = zmax / 100  # Typical cutoff height
k = 1 / l  # Decay factor
alpha = tau * k / (1 - np.exp(-k * zmax))  # Pre-exponential factor
print(f"{alpha = }")

z = np.linspace(0, zmax, n)
atmosphere_data = np.reshape(alpha * np.exp(-k * z), (-1, 1, 1))
plt.plot(atmosphere_data.squeeze())

dz = zmax / (n - 1)
print(f"tau = {np.sum(atmosphere_data) * dz}")

In [ ]:
scene_dict_ref = {
    "type": "scene",
    # "atmosphere_medium": {
    #     "type": "homogeneous",
    #     "sigma_t": 1.0,
    #     "albedo": 0.5,
    #     "phase": {"type": "rayleigh"},
    # },
    "atmosphere_medium": {
        "type": "heterogeneous",
        "sigma_t": {
            "type": "gridvolume",
            "grid": mi.VolumeGrid(atmosphere_data),
            "accel": False,
        },
        "albedo": 0.5,
        # "majorant_resolution_factor": [1, 1, 100],
        "phase": {"type": "rayleigh"},
    },
    "atmosphere_shape": {
        "type": "cube",
        "interior": {"type": "ref", "id": "atmosphere_medium"},
        "to_world": mi.ScalarTransform4f.translate([0.5, 0.5, 0.5])
        @ mi.ScalarTransform4f.scale(0.5),
        "bsdf": {"type": "null"},
    },
    "surface": {
        "type": "rectangle",
        "to_world": mi.ScalarTransform4f.translate([0.5, 0.5, 0.0])
        @ mi.ScalarTransform4f.scale(0.5),
        "bsdf": {
            "type": "diffuse",
            "reflectance": {
                "type": "checkerboard",
                "to_uv": mi.ScalarTransform4f.scale(2),
            },
        },
    },
    "camera": {
        "type": "perspective",
        "to_world": mi.ScalarTransform4f.look_at(
            origin=[0.5, -2, 0.5],
            target=[0.5, 0.5, 0.5],
            up=[0, 0, 1],
        ),
        "sampler": {"type": "independent"},
        "film": {
            "type": "hdrfilm",
            "width": 256,
            "height": 256,
            "rfilter": {"type": "box"},
        },
        "near_clip": 1e-8,
        "far_clip": 1e8,
    },
    "illumination": {
        "type": "directional",
        "direction": [0, -1, -1],
        "irradiance": 5.0,
    },
    "integrator": {"type": "volpath"},
}
scene_ref = mi.load_dict(scene_dict_ref)

scene_dict_maj = scene_dict_ref.copy()
scene_dict_maj["atmosphere_medium"] = {
    **scene_dict_maj["atmosphere_medium"].copy(),
    **{"majorant_resolution_factor": [1, 1, 64]},
}
scene_maj = mi.load_dict(scene_dict_maj)

In [ ]:
mi_logger.set_log_level(mi.LogLevel.Info)

def print_interaction(mei):
    print("MediumInteraction[")
    print(f"  t = {mei.t},")
    print(f"  time = {mei.time},")
    print(f"  wavelengths = {mei.wavelengths},")
    print(f"  p = {mei.p},")
    print(f"  wi = {mei.wi},")
    print(f"  combined_extinction = {mei.combined_extinction},")
    print("]")

medium_ref = scene_ref.shapes()[0].interior_medium()
medium_maj = scene_maj.shapes()[0].interior_medium()

ray = dr.zeros(mi.Ray3f)
ray.o = mi.ScalarPoint3f([0.5, 0.5, 0.0])
ray.d = mi.ScalarVector3f([0, 0, 1])
ray.maxt = 1e5

print(ray)
sample = 0.5
print_interaction(medium_maj.sample_interaction(ray, sample, 0, True))
print_interaction(medium_ref.sample_interaction(ray, sample, 0, True))

In [ ]:
# Visualize scene data
try:
    params = mi.traverse(scene_maj.shapes()[0].interior_medium())
    sigma_t = np.squeeze(params["sigma_t.data"])
    majorant = np.squeeze(params["majorant_grid.data"])
    plt.plot(np.linspace(0, 1, len(sigma_t)), sigma_t, label="$\sigma_t$")
    plt.step(np.linspace(0, 1, len(majorant)), majorant, where="mid", label="Majorant")
    plt.xlabel("z")
    plt.ylabel("[m$^{-1}$]")
    plt.legend()
except KeyError:
    print("No majorant grid found")

In [ ]:
mi_logger.set_log_level(mi.LogLevel.Warn)

spp = 4096  # For an accurate comparison
spp = 256  # For a quick benchmark

print("Global majorant")
%time arr_ref = mi.render(scene_ref, spp=spp).numpy()
display(mi.Bitmap(arr_ref))

print("Local majorant grid")
%time arr_maj = mi.render(scene_maj, spp=spp).numpy()
display(mi.Bitmap(arr_maj))

In [ ]:
import matplotlib as mpl
import matplotlib.colors as colors

with np.errstate(divide="ignore", invalid="ignore"):
    arr_diff = np.abs((arr_maj - arr_ref) / arr_ref)

arr_diff_masked = np.ma.array(arr_diff, mask=~np.isfinite(arr_diff))
diff_vmin = arr_diff_masked.min()
diff_vmax = arr_diff_masked.max()

fig, axs = plt.subplots(3, 3, figsize=(12, 12), layout="constrained")

for i_row, cmap, channel_name in zip(
    range(3),
    # ["Reds_r", "Greens_r", "Blues_r"],
    ["viridis"] * 3,
    ["Red", "Green", "Blue"],
):
    cmap = mpl.cm.get_cmap(cmap).copy()
    cmap.set_bad("black")

    for i_col, data, title, norm, vlim in zip(
        range(3),
        [arr_ref, arr_maj, arr_diff * 100],
        ["Global majorant", "Local majorant grid", "Relative difference [%]"],
        [None, None, "log"],
        [None, None, [1e-3, 1e0]],
    ):
        ax = axs[i_row, i_col]
        if i_row == 0:
            ax.set_title(title)
        if i_col == 0:
            ax.annotate(
                channel_name,
                xy=(0, 0.5),
                xytext=(-0.1, 0.5),
                textcoords="axes fraction",
                ha="left",
                va="center",
                rotation=90,
                size=20,
            )

        kwargs = {}

        if norm == "log":
            (vmin, vmax) = vlim if vlim else (diff_vmin, diff_vmax)

            kwargs["norm"] = colors.LogNorm(vmin=vmin, vmax=vmax)

        else:
            if vlim:
                kwargs["vmin"] = vlim[0]
                kwargs["vmax"] = vlim[1]

        ax.axis("off")
        cm = ax.imshow(data[:, :, i_row], cmap=cmap, **kwargs)
        plt.colorbar(cm, ax=ax, shrink=0.74)